In [1]:
#import necessary modules
from qpython.qconnection import QConnection as qcon
from qpython.qcollection import QDictionary as qdict
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#set qcon variables
host='localhost'
un='admin'
pswd='admin'

In [2]:
#run torq summary
torq_sum= ! "$(cd "$(dirname "${BASH_SOURCE[0]}")" && pwd)"/torq.sh summary

#convert array to numpy array
summary=np.asarray(torq_sum)
summary

array(['TIME     | PROCESS        | STATUS | PID    | PORT  ',
       '14:07:31 | discovery1     | up     | 4047   | 1701  ',
       '14:07:31 | tickerplant1   | up     | 4139   | 1700  ',
       '14:07:31 | rdb1           | down   |',
       '14:07:31 | hdb1           | up     | 21064  | 1703  ',
       '14:07:31 | hdb2           | up     | 4413   | 1704  ',
       '14:07:31 | wdb1           | up     | 4505   | 1705  ',
       '14:07:31 | sort1          | up     | 4597   | 1706  ',
       '14:07:31 | gateway1       | up     | 4689   | 1707  ',
       '14:07:31 | killtick       | down   |',
       '14:07:31 | monitor1       | up     | 4781   | 1709  ',
       '14:07:31 | tpreplay1      | down   |',
       '14:07:31 | housekeeping1  | up     | 4873   | 1711  ',
       '14:07:31 | reporter1      | up     | 4965   | 1712  ',
       '14:07:31 | compression1   | down   |',
       '14:07:31 | feed1          | up     | 5147   | 1714  ',
       '14:07:31 | chainedtp1     | up     | 5239   | 17

In [3]:
#split each element of array by | character
sum_list = [i.split('|') for i in summary]
df = pd.DataFrame(sum_list)

# take first row and use as header for df
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

#trim whitespace from headers
cols=[]
for i in df.columns:
    cols.append(str.strip(i))
df.columns=cols

#trim whitespace from all objects within dataframe
data = df.select_dtypes(['object'])
df[data.columns] = df.apply(lambda x: x.str.strip())

In [4]:
# function to extract the port number for each process - takes a string argument
def find_portno(process):
    process_info = df.loc[df['PROCESS'] == process]
    PORT = process_info['PORT'].astype(str).astype(int)
    return PORT

In [5]:
#function to decode bytes to strings
def byte_decode(table,cols):
    table[cols] = table[cols].applymap(lambda x: x.decode('utf-8'))

In [6]:
# set colour on down processes to red and up processes to green
def colour_down_red(col):
    color = 'red' if 'down' in col else 'green'
    return 'color: %s' % color
df.style.applymap(colour_down_red, subset=['STATUS'])

,TIME,PROCESS,STATUS,PID,PORT
1,14:07:31,discovery1,up,4047,1701
2,14:07:31,tickerplant1,up,4139,1700
3,14:07:31,rdb1,down,,None
4,14:07:31,hdb1,up,21064,1703
5,14:07:31,hdb2,up,4413,1704
6,14:07:31,wdb1,up,4505,1705
7,14:07:31,sort1,up,4597,1706
8,14:07:31,gateway1,up,4689,1707
9,14:07:31,killtick,down,,None
10,14:07:31,monitor1,up,4781,1709


In [12]:
with qcon(host, port=find_portno('tickerplant1'), username=un, password=pswd,timeout=3.0) as q:
    counts = q(".u.i")
counts

72686